# 03. Text2SQL 기초

이 노트북에서 다루는 내용:
- Text2SQL 개념 이해
- 자연어로부터 SQL 생성
- 생성된 쿼리(Query) 실행
- 오류 처리 및 검증
- 쿼리 히스토리 로깅

## 1. 라이브러리 임포트

In [ ]:
import sys
sys.path.append('/workspace')

from src.utils.db_utils import DatabaseConnection, get_database_context
from src.utils.text2sql_utils import Text2SQLGenerator, execute_text2sql, get_few_shot_examples
import pandas as pd

print("✓ Libraries imported successfully")

## 2. 컴포넌트 초기화

In [ ]:
# 데이터베이스 초기화 connection
db = DatabaseConnection()

# Text2SQL 생성기 초기화
try:
    generator = Text2SQLGenerator(llm_provider="ollama", model_name="llama2")
    print("✓ Text2SQL generator initialized")
except Exception as e:
    print(f"⚠ Could not initialize Text2SQL generator: {e}")
    print("Make sure Ollama is running and a model is installed")
    print("Run: docker exec -it text2sql-ollama ollama pull llama2")

## 3. 데이터베이스 스키마 컨텍스트 가져오기

In [ ]:
# 컨텍스트를 위한 데이터베이스 스키마 가져오기
schema_context = get_database_context()
print("Database Schema Context:")
print("="*60)
print(schema_context)

## 4. Few-Shot 예제 검토

In [ ]:
# LLM이 작업을 이해하는 데 도움이 되는 Few-Shot 예제 가져오기
examples = get_few_shot_examples()
print("Few-Shot Examples:")
print("="*60)
print(examples)

## 5. 간단한 Text2SQL 예제

In [ ]:
# 예제 1: 간단한 SELECT 쿼리
natural_query = "Show me all employees"

print(f"Natural Language Query: {natural_query}")
print("\nGenerating SQL...")

try:
    sql_query = generator.generate_sql(natural_query, schema_context, examples)
    print(f"\nGenerated SQL:\n{sql_query}")
    
    # 쿼리 실행
    result_df = db.execute_query_df(sql_query)
    print(f"\nResults ({len(result_df)} rows):")
    display(result_df)
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# 예제 2: 필터링이 있는 쿼리
natural_query = "Find all employees with salary greater than 6000000"

print(f"Natural Language Query: {natural_query}")
print("\nGenerating SQL...")

try:
    sql_query = generator.generate_sql(natural_query, schema_context, examples)
    print(f"\nGenerated SQL:\n{sql_query}")
    
    result_df = db.execute_query_df(sql_query)
    print(f"\nResults ({len(result_df)} rows):")
    display(result_df)
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# 예제 3: JOIN이 있는 쿼리
natural_query = "Show me all employees in the Engineering department"

print(f"Natural Language Query: {natural_query}")
print("\nGenerating SQL...")

try:
    sql_query = generator.generate_sql(natural_query, schema_context, examples)
    print(f"\nGenerated SQL:\n{sql_query}")
    
    result_df = db.execute_query_df(sql_query)
    print(f"\nResults ({len(result_df)} rows):")
    display(result_df)
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# 예제 4: 집계 쿼리
natural_query = "What is the total sales amount by region?"

print(f"Natural Language Query: {natural_query}")
print("\nGenerating SQL...")

try:
    sql_query = generator.generate_sql(natural_query, schema_context, examples)
    print(f"\nGenerated SQL:\n{sql_query}")
    
    result_df = db.execute_query_df(sql_query)
    print(f"\nResults ({len(result_df)} rows):")
    display(result_df)
except Exception as e:
    print(f"Error: {e}")

## 6. 완전한 파이프라인 사용

In [ ]:
# 로깅과 함께 완전한 파이프라인 사용
natural_query = "List all active projects with their department names"

print(f"Natural Language Query: {natural_query}")
print("\nExecuting complete text2sql pipeline...")

result = execute_text2sql(db, natural_query, log_execution=True)

print(f"\nSuccess: {result['success']}")
print(f"Execution Time: {result['execution_time_ms']} ms")
print(f"\nGenerated SQL:\n{result['sql_query']}")

if result['success']:
    print(f"\nResults ({result['row_count']} rows):")
    display(result['results'])
else:
    print(f"\nError: {result['error']}")

## 7. 직접 쿼리 시도해보기

In [ ]:
# 인터랙티브 쿼리 테스트
test_queries = [
    "How many employees are in each department?",
    "What is the average salary by job title?",
    "Show me the top 5 customers by total purchase amount",
    "List all projects that ended in 2023",
    "Find employees who are managers"
]

# 각 쿼리 테스트
for query in test_queries:
    print("\n" + "="*80)
    print(f"Query: {query}")
    print("="*80)
    
    result = execute_text2sql(db, query, log_execution=True)
    
    if result['success']:
        print(f"Generated SQL: {result['sql_query']}")
        print(f"\nResults ({result['row_count']} rows):")
        display(result['results'].head())
    else:
        print(f"Error: {result['error']}")

## 8. 쿼리 히스토리 보기

In [ ]:
# 쿼리 히스토리 확인
history_query = """
SELECT 
    query_id,
    natural_language_query,
    execution_success,
    execution_time_ms,
    result_count,
    created_at
FROM query_history
ORDER BY created_at DESC
LIMIT 10
"""

history_df = db.execute_query_df(history_query)
print("Recent Query History:")
display(history_df)

In [ ]:
# 쿼리 성공 통계
stats_query = """
SELECT 
    COUNT(*) as total_queries,
    SUM(CASE WHEN execution_success THEN 1 ELSE 0 END) as successful,
    SUM(CASE WHEN NOT execution_success THEN 1 ELSE 0 END) as failed,
    ROUND(AVG(execution_time_ms), 2) as avg_execution_time_ms,
    ROUND(AVG(result_count), 2) as avg_result_count
FROM query_history
"""

stats_df = db.execute_query_df(stats_query)
print("Query Statistics:")
display(stats_df)

## 9. 오류 처리 예제

In [ ]:
# 실패할 수 있는 쿼리 시도
ambiguous_query = "Show me the data"

print(f"Testing ambiguous query: {ambiguous_query}")
result = execute_text2sql(db, ambiguous_query, log_execution=True)

print(f"\nSuccess: {result['success']}")
if result['success']:
    print(f"Generated SQL: {result['sql_query']}")
    display(result['results'].head())
else:
    print(f"Error: {result['error']}")
    print("\n💡 Tip: Be more specific in your query!")

## 요약

이 노트북에서 배운 내용:
- ✓ 자연어 쿼리로부터 SQL 생성 방법
- ✓ 스키마 컨텍스트와 Few-Shot 예제 사용 방법
- ✓ 생성된 쿼리 실행 및 검증 방법
- ✓ 모니터링을 위한 쿼리 로깅 방법
- ✓ Text2SQL에서 오류 처리 방법

다음 단계: `04_agent_workflow.ipynb`로 이동하여 LangChain/LangGraph로 에이전트 워크플로우를 구축합니다.